# Development Environment and Permissions 

## Installation

_*Note:* we only install the required libraries from Hugging Face and AWS. You also need PyTorch or Tensorflow, if you haven´t it installed_

In [1]:
%%writefile requirements.txt

ipywidgets
git+https://github.com/huggingface/transformers
datasets
sacrebleu
torch
sentencepiece
evaluate

Overwriting requirements.txt


In [1]:
%%capture
%pip install -r requirements.txt

In [2]:
%%capture
!pip install accelerate -U

In [1]:
# %%capture
# %pip install -r colab_requirements.txt

## Permissions

_If you are going to use Sagemaker in a local environment. You need access to an IAM Role with the required permissions for Sagemaker. You can find [here](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html) more about it._

In [4]:
# pandas==1.5.2
# s3fs==2022.11.0

In [3]:
import sagemaker

sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket='mangago-dataset'
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::984909470121:role/service-role/AmazonSageMakerServiceCatalogProductsUseRole
sagemaker bucket: mangago-dataset
sagemaker session region: us-east-1


# Preprocessing

In [4]:
import botocore
from datasets.filesystems import S3FileSystem

s3 = S3FileSystem()

bucket ='mangago-dataset'
folder = 'raw'
data_key = 'deepl_translated_df_47.csv'
data_location = 's3://{}/{}/{}'.format(bucket, folder, data_key)

In [5]:
data_location

's3://mangago-dataset/raw/deepl_translated_df_47.csv'

In [6]:
import pandas as pd

df = pd.read_csv('s3://mangago-dataset/raw/deepl_translated_df_47.csv', sep=';', encoding='utf-8')
df = df.rename(columns={'text':'ja','translation':'fr'})

In [7]:
import datasets
from datasets import Dataset, DatasetDict, load_dataset, load_metric

csv_dataset = datasets.load_dataset("csv", data_files=data_location, keep_default_na=False, delimiter=';')
dataset = csv_dataset['train']

# dataset = Dataset.from_pandas(df)

dataset = dataset.rename_columns({'text': 'ja', 'translation': 'fr'})

# 90% train, 10% (test + validation)
dataset_train_test = dataset.train_test_split(test_size=0.1, seed=42)

# 10% to 5% test + 5% validation
dataset_test_valid = dataset_train_test['test'].train_test_split(test_size=0.5, seed=42)

dataset = datasets.DatasetDict({
    'train': dataset_train_test['train'],
    'test': dataset_test_valid['test'],
    'validation': dataset_test_valid['train']})

Found cached dataset csv (/home/ec2-user/.cache/huggingface/datasets/csv/default-7903a7d381a39996/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached split indices for dataset at /home/ec2-user/.cache/huggingface/datasets/csv/default-7903a7d381a39996/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-a8fbf5b04ded70ea.arrow and /home/ec2-user/.cache/huggingface/datasets/csv/default-7903a7d381a39996/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-5fb917b197b8cd09.arrow
Loading cached split indices for dataset at /home/ec2-user/.cache/huggingface/datasets/csv/default-7903a7d381a39996/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-9e7a853b76bc146c.arrow and /home/ec2-user/.cache/huggingface/datasets/csv/default-7903a7d381a39996/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-11c934a67949fffd.arrow


## Tokenization 

In [8]:
from transformers import AutoTokenizer

tokenizer_name = 'Helsinki-NLP/opus-mt-ja-fr'
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [9]:
prefix = ""
max_input_length = 256
max_target_length = 256
source_lang = "ja"
target_lang = "fr"

def preprocess_function(examples):
    inputs = [prefix + ex for ex in examples["ja"]]
    targets = [ex for ex in examples["fr"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [10]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Loading cached processed dataset at /home/ec2-user/.cache/huggingface/datasets/csv/default-7903a7d381a39996/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-919eb33527a1168a.arrow
Loading cached processed dataset at /home/ec2-user/.cache/huggingface/datasets/csv/default-7903a7d381a39996/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-6db5a8cd7dd3a702.arrow
Loading cached processed dataset at /home/ec2-user/.cache/huggingface/datasets/csv/default-7903a7d381a39996/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-6fb7bcc646b8cc32.arrow


In [ ]:
from datetime import datetime

save_path = f's3://{bucket}/preprocessed/dataset-{datetime.now().strftime("%d-%m-%Y-H-M")}'
tokenized_datasets.save_to_disk(save_path, fs=s3)

In [11]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

Loading cached shuffled indices for dataset at /home/ec2-user/.cache/huggingface/datasets/csv/default-7903a7d381a39996/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-c1319dc038baae5c.arrow
Loading cached shuffled indices for dataset at /home/ec2-user/.cache/huggingface/datasets/csv/default-7903a7d381a39996/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-972139eecc90b079.arrow


In [15]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

# model = AutoModelForSeq2SeqLM.from_pretrained("checkpoint-36500", from_tf=True)
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ja-fr")

In [16]:
from datetime import datetime

batch_size = 16
args = Seq2SeqTrainingArguments(
    f"model-finetuned-{datetime.now().strftime('%d-%m-%Y')}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=7,
    save_strategy = "no",
    load_best_model_at_end=False,
    predict_with_generate=True
)

In [17]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [21]:
import numpy as np

metric = load_metric("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    return result

/tmp/ipykernel_7418/365506015.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")


In [22]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [23]:
trainer.train()

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,No log,2.125080,5.366800,27.916000
2,No log,2.082277,6.326300,27.654000
3,No log,2.066849,7.843300,24.103000
4,No log,2.060434,9.301800,22.124000
5,No log,2.058435,9.164400,22.055000


TrainOutput(global_step=315, training_loss=1.6601982964409723, metrics={'train_runtime': 688.3966, 'train_samples_per_second': 7.263, 'train_steps_per_second': 0.458, 'total_flos': 34678027321344.0, 'train_loss': 1.6601982964409723, 'epoch': 5.0})

In [ ]:
from datetime import datetime
output_model_name = f"model_finetuned_{datetime.now().strftime('%d-%m-%Y')}.tar.gz"

In [27]:
# trainer.save_model('model/')

In [61]:
import tarfile
import os

def compress(tar_dir=None,output_file=output_model_name):
    parent_dir=os.getcwd()
    os.chdir(tar_dir)
    with tarfile.open(os.path.join(parent_dir, output_file), "w:gz") as tar:
        for item in os.listdir('.'):
          print(item)
          tar.add(item, arcname=item)
    os.chdir(parent_dir)

compress(str('model_finetuned/'))

config.json
vocab.json
pytorch_model.bin
source.spm
special_tokens_map.json
training_args.bin
target.spm
generation_config.json
tokenizer_config.json


In [62]:
from sagemaker.s3 import S3Uploader

s3_model_uri = S3Uploader.upload(local_path=output_model_name, desired_s3_uri=f"s3://mangago-finetuned-models/model_finetuned_{datetime.now().strftime('%d-%m-%Y')}/")
print(f"model uploaded to: {s3_model_uri}")

model uploaded to: s3://mangago-finetuned-models/model_finetuned_02-07-2023//model_finetuned_02-07-2023.tar.gz


In [60]:
trainer.save_model('s3://mangago-finetuned-models/model-finetuned-02-07/')

## Compare both models

In [13]:
ja_valid = [
     'だからっ',
     '知らないって言ってるだろっ',
     'そんな借金なんて!',
     'そうは言ってもなぁ',
     'こっちにゃ借用書があんだよ',
     'トルティヤーノに借りた金はちゃんと返して貰わねぇと',
     '知るもんかっ',
     '父親がカジノで作った借金なんて...',
     'もしやトルティヤーノ一家の方でしょうか',
     '取り込み中だ出直しやがれ!',
     '待ってくださーー',
     'すいません...っ目の見えない「閉じ眼」なものでつい転んでじゃって',
     '閉じ眼がマフィアに何の用だコラァ!!',
     '流しのガンマンをしているシオラと申します',
     'ここら一帯を仕切るトルティヤー一家の名が聞こえたので',
     '用心棒として雇っていただけないかと...',
     '目の見えない閉じ眼が...',
     'ガンマンだって...?',
     'マフィア舐めんのも大概にしやがれ',
     '閉じ眼が用心棒だ?',
     '腰の銃でイかれた脳みそぶちまけてみるか?',
     'ちょっとやめてよ!こんなとこで!',
     'お前もしかして',
     '女か?',
     'えっはい...そうですけど...',
     'ほぉ',
     '面見せてみろ',
     '閉じ眼でも器量が良けりゃ貿い手は...',
     'やっちょっと...!',
]

fr_valid = [
     'Je vous le dis !',
     'Je ne sais pas de quoi vous parlez !',
     'je ne vous dois rien !',
     'Eh bien, je suis désolé...',
     'nous avons des preuves ici...',
     'Vous savez que nous avons besoin que vous nous remboursiez...',
     'Comment le saurais-je ?',
     'Comment puis-je connaître la dette de mon père dans les casinos...',
     'Vous êtes membre de la famille Tortillano ?',
     'nous avons une conversation ici ! Perdez la tête !',
     "S'il vous plaît... ! Attendez...",
     'Je suis désolé, je ne vois rien, je suis un "œil fermé"...',
     "Alors, qu'est-ce qu'un œil fermé attend de nous ?",
     'Je suis Siora, un tireur',
     "J'ai entendu dire que c'est vous, les Tortillanos, qui êtes aux commandes ici.",
     "Je me demandais si vous pouviez m'engager comme garde du corps...",
     'donc un œil fermé est...',
     'un tireur... ?',
     'Ne vous moquez pas de nous.',
     'yeux fermés pour un garde du corps ?',
     'Tu veux que je te tire une balle dans la tête ? !',
     "Arrêtez, s'il vous plaît !",
     'êtes-vous...',
     'une fille ?',
     'oui, mais alors... ?',
     'oh bien sûr...',
     'Regardons votre visage.',
     'Si vous avez de bonnes qualités, vos yeux ne sont pas un problème.',
     'Hé, attendez !',
]

In [6]:
from transformers import pipeline

translator = pipeline("translation", model='Helsinki-NLP/opus-mt-ja-fr')
translator('面見せてみろ')

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


[{'translation_text': 'Fais-moi voir.'}]

In [11]:
from transformers import MarianMTModel, MarianTokenizer, AutoTokenizer, AutoModelForSeq2SeqLM
model_name = 'model_nul/'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained('model_nul/')

translated = model.generate(**tokenizer('閉じ眼でも器量が良けりゃ貿い手は...', return_tensors="pt", padding=True), repetition_penalty=10.)
[tokenizer.decode(t, skip_special_tokens=True) for t in translated]

["Si le volume de l'œil est bon, il y a une main lourde..."]

In [33]:
basic_model_name = 'Helsinki-NLP/opus-mt-ja-fr'
finetuned_model_name = 'model_nul/'

basic_model = AutoModelForSeq2SeqLM.from_pretrained(basic_model_name)
finetuned_model = AutoModelForSeq2SeqLM.from_pretrained(finetuned_model_name)

basic_tokenizer = AutoTokenizer.from_pretrained(basic_model_name)
finetuned_tokenizer = AutoTokenizer.from_pretrained(finetuned_model_name)
    
basic_generation = basic_model.generate(**tokenizer(ja_valid[:10], return_tensors="pt", padding=True), repetition_penalty=10.)
basic_translations = [basic_tokenizer.decode(t, skip_special_tokens=True) for t in basic_generation]

finetuned_generation = finetuned_model.generate(**tokenizer(ja_valid[:10], return_tensors="pt", padding=True), repetition_penalty=10.)
finetuned_translations = [finetuned_tokenizer.decode(t, skip_special_tokens=True) for t in finetuned_generation]

In [34]:
basic_translations

["C'est pour ça que...",
 "Je te l'ai dit, je ne sais pas.",
 "Je n'arrive pas à croire que tu me dois ça!",
 "Je n'ai pas dit ça.",
 "J'ai une carte de crédit.",
 "Il a dit qu'il n'avait pas besoin de récupérer l'argent que j'avais emprunté à Trujano.",
 'Je ne sais pas.',
 'Je ne peux pas croire que mon père ait créé une dette dans un casino.',
 "C'est la famille Turtiliano.",
 "Je suis en train de m'en occuper. Rebrousse-la!"]

In [37]:
finetuned_translations

["C'est pour cela que...",
 'Je vous dis que je ne sais pas.',
 "Je n'arrive pas à croire que vous ayez une telle dette!",
 "Je n'ai pas dit ça.",
 "J'ai un compte de dépôt ici.",
 "L'argent que j'ai prêté à Trujano, je dois le rendre.",
 'Je ne sais pas.',
 "Je n'arrive pas à croire que mon père ait créé une dette dans un casino.",
 "Si c'est le cas, est-ce la famille Turtiliano?",
 'Je suis en train de le faire. Répare-toi!']

In [36]:
from sacrebleu import corpus_bleu

basic_model_bleu = corpus_bleu(basic_translations, fr_valid[:10])
finetuned_model_bleu = corpus_bleu(finetuned_translations, fr_valid[:10])

print("basic_model_bleu:", basic_model_bleu.score)
print("finetuned_model_bleu:", finetuned_model_bleu.score)

basic_model_bleu: 0.46411279345926965
finetuned_model_bleu: 0.0


In [40]:
fr_valid[:10]

['Je vous le dis !',
 'Je ne sais pas de quoi vous parlez !',
 'je ne vous dois rien !',
 'Eh bien, je suis désolé...',
 'nous avons des preuves ici...',
 'Vous savez que nous avons besoin que vous nous remboursiez...',
 'Comment le saurais-je ?',
 'Comment puis-je connaître la dette de mon père dans les casinos...',
 'Vous êtes membre de la famille Tortillano ?',
 'nous avons une conversation ici ! Perdez la tête !']

In [38]:
corpus_bleu(finetuned_translations, fr_valid[:10])

BLEU = 0.00 0.0/0.0/0.0/0.0 (BP = 1.000 ratio = 9.400 hyp_len = 94 ref_len = 10)

In [39]:
corpus_bleu(basic_translations, fr_valid[:10])

BLEU = 0.46 1.1/0.6/0.3/0.2 (BP = 1.000 ratio = 9.200 hyp_len = 92 ref_len = 10)

In [42]:
import evaluate

bleu = evaluate.load("bleu")
print(bleu.compute(predictions=finetuned_translations, references=fr_valid[:10]))

{'bleu': 0.0, 'precisions': [0.22340425531914893, 0.05952380952380952, 0.013513513513513514, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 1.0930232558139534, 'translation_length': 94, 'reference_length': 86}


In [49]:
print(bleu.compute(predictions=basic_translations, references=fr_valid[:10])['bleu'])

0.0


In [44]:
fr_valid[:10]

['Je vous le dis !',
 'Je ne sais pas de quoi vous parlez !',
 'je ne vous dois rien !',
 'Eh bien, je suis désolé...',
 'nous avons des preuves ici...',
 'Vous savez que nous avons besoin que vous nous remboursiez...',
 'Comment le saurais-je ?',
 'Comment puis-je connaître la dette de mon père dans les casinos...',
 'Vous êtes membre de la famille Tortillano ?',
 'nous avons une conversation ici ! Perdez la tête !']

In [50]:
basic_translations

["C'est pour ça que...",
 "Je te l'ai dit, je ne sais pas.",
 "Je n'arrive pas à croire que tu me dois ça!",
 "Je n'ai pas dit ça.",
 "J'ai une carte de crédit.",
 "Il a dit qu'il n'avait pas besoin de récupérer l'argent que j'avais emprunté à Trujano.",
 'Je ne sais pas.',
 'Je ne peux pas croire que mon père ait créé une dette dans un casino.',
 "C'est la famille Turtiliano.",
 "Je suis en train de m'en occuper. Rebrousse-la!"]